# Results visualisation

In this notebook ve visualize the results of sigma test and show the effects of test time adaptation.

In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import torch
import torch.optim as optim
import json
from PIL import Image
import copy

import os
os.chdir("..")

from model import CountRegressor, Resnet50FPN
from utils import MAPS, Scales, Transform, extract_features
from utils import MincountLoss, PerturbationLoss

## Sigma experiment

In [4]:
f = open("data/test_reproduced/sigma_selection.txt", "r")
sigmas = list(range(2, 22, 2))
MAE, SD = [], []

state = 0
for line in f:
    if state == 2:
        MAE.append(float(line.strip().split(" ")[1]))
    elif state == 3:
        SD.append(float(line.strip().split(" ")[1]))

    state = (state + 1) % 7

n = 1286
SE = [s / np.sqrt(n) for s in SD]

plt.plot(sigmas, MAE, color="b", marker="o")
plt.plot(sigmas, [ae + e for (ae, e) in zip(MAE, SE)], color="b", ls="--")
plt.plot(sigmas, [ae - e for (ae, e) in zip(MAE, SE)], color="b", ls="--")
plt.xlabel("Sigma")
plt.ylabel("MAE")
plt.title("Effect of perturbation sigma on MAE")
plt.savefig("data/test_reproduced/sigma.png")
%matplotlib inline
plt.show()

## Adaption check

In [2]:
resnet50_conv = Resnet50FPN()
resnet50_conv.cuda()
resnet50_conv.eval()

regressor = CountRegressor(6, pool='mean')
regressor.load_state_dict(torch.load("data/pretrainedModels/FamNet_Save1.pth"))
regressor.cuda()
regressor.eval()

with open("data/annotation_FSC147_384.json") as f:
    annotations = json.load(f)

In [9]:
f = open("data/test_reproduced/adapt_change.txt")
f_out = open("data/test_reproduced/adapt_predictions.txt", "w")

for line in f:
    if line[0] == "3":
        continue
    im_id = line.split(": ")[0]
    f_out.write(f"Image: {im_id}\n")

    anno = annotations[im_id]
    bboxes = anno['box_examples_coordinates']
    dots = np.array(anno['points'])

    rects = list()
    for bbox in bboxes:
        x1, y1 = bbox[0][0], bbox[0][1]
        x2, y2 = bbox[2][0], bbox[2][1]
        rects.append([y1, x1, y2, x2])

    image = Image.open("data/images_384_VarV2/" + im_id)
    image.load()
    sample = {'image': image, 'lines_boxes': rects}
    sample = Transform(sample)
    image, boxes = sample['image'], sample['boxes']

    image = image.cuda()
    boxes = boxes.cuda()

    with torch.no_grad(): features = extract_features(resnet50_conv, image.unsqueeze(0), boxes.unsqueeze(0), MAPS, Scales)

    # Without adaptation
    with torch.no_grad(): output = regressor(features)

    gt_cnt = dots.shape[0]
    f_out.write(f"Ground truth: {gt_cnt}\n")
    pred_cnt = output.sum().item()
    f_out.write(f"Predicted: {pred_cnt}\n")

    output = output.squeeze(0).to('cpu').numpy()[0].astype("float32")
    output = Image.fromarray(np.uint8(cm.viridis(output/np.max(output))*255))
    output.save("data/test_reproduced/adaptation_predictions/" + im_id.rstrip("jpg") + "png")

    # With adaptation
    features.required_grad = True
    adapted_regressor = copy.deepcopy(regressor)
    adapted_regressor.train()
    optimizer = optim.Adam(adapted_regressor.parameters(), lr=1e-7)
    for step in range(0, 100):
        optimizer.zero_grad()
        output = adapted_regressor(features)
        lCount = 1e-9 * MincountLoss(output, boxes)
        lPerturbation = 1e-4 * PerturbationLoss(output, boxes, sigma=8)
        Loss = lCount + lPerturbation
        # loss can become zero in some cases, where loss is a 0 valued scalar and not a tensor
        # So Perform gradient descent only for non zero cases
        if torch.is_tensor(Loss):
            Loss.backward()
            optimizer.step()
    features.required_grad = False
    output_adapt = adapted_regressor(features)

    pred_cnt = output_adapt.sum().item()
    f_out.write(f"Predicted with adaptation: {pred_cnt}\n")

    output_adapt = output_adapt.squeeze(0).to('cpu').detach().numpy()[0].astype("float32")
    output_adapt = Image.fromarray(np.uint8(cm.viridis(output_adapt/np.max(output_adapt))*255))
    output_adapt.save("data/test_reproduced/adaptation_predictions/" + im_id.rstrip(".jpg") + "_adapt.png")

f.close()
f_out.close()

/home/domenv/conda/envs/reproducibility/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/home/domenv/MLDS2/src/utils.py:106: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  Loss += F.mse_loss(X,ones)
